In [ ]:
#lets start here

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import math
from bs4 import BeautifulSoup
import requests as req
import datetime
from datetime import timedelta  
import json

%matplotlib inline

In [ ]:
lga = pd.read_csv("LGA Postcode.csv")
#print(lga)
# Let's load the station information
station = pd.read_csv("StationEntrances2020_v4.csv", header=0)
#
# Add Radian since quicker later
station['LATR'] = station['LAT'] * math.pi / 180.0
station['LONGR'] = station['LONG'] * math.pi / 180.0

In [ ]:
def findlenjson(t):
    #Need to find end of JSON
    c = 1
    brackets = 1
    while (brackets != 0 and t[c]):
        xo = t[c:].find('{')
        xc = t[c:].find('}')
        if (xc < xo):
            brackets -= 1
            c += xc+1
        else:
            brackets += 1
            c += xo + 1
    return c


def findregion(x):
    try:
        r = lga[lga["PostCode"] == x]
        r = r.iloc[0]["Council Name"]
    except:
        r = np.NaN
    return r
   

def processpage(resp):
    w = pd.DataFrame()
    jsonstart = resp.text.find("componentProps")
    if (jsonstart == -1):
        return w
    jsonstart +=16
    print(date.strftime("%Y-%m-%d"), "Processing")
    #print("Found JSON: ", resp.text[jsonstart:jsonstart+20])
   # print("text length ", resp.text.len())

    #Need to find end of JSON
    c = jsonstart + findlenjson(resp.text[jsonstart:])
    
    dict = json.loads(resp.text[jsonstart:c])
   
    for i in dict["salesListings"]:
        for j in i['listings']:
    #        print(j)
            try:
                v = pd.DataFrame(j)
    #            print(v[0])
                w = pd.concat([w, v], sort=False)
            except:
                ui=1

#    print(w)
    w["lat"] = np.NaN
    w["long"] = np.NaN
    w["station"] = np.NaN
    w["station_name"] = ""
    w["central_station"] = np.NaN
    
    for i in np.arange(w.shape[0],step=2):
        print(i)
        w["lat"][i] = w["geoLocation"][i]
        w["long"][i] = w["geoLocation"][i+1]
        #dfmi.loc[:, ('one', 'second')]
        
        
        #w.iloc[:,(i,"lat")] = w["geoLocation"][i]
        #w.iloc[i,"long"] = w["geoLocation"][i+1]
        
        #dfd.loc[2, 'a'] = row, col
        #w.loc[:,(i,'lat')] = w["geoLocation"][i]
        #w.loc[:,(i,'long')] = w["geoLocation"][i+1]

        #w.loc[:,('lat',i)] = w["geoLocation"][i]
        #w.loc[:,('long',i)] = w["geoLocation"][i+1]

        lat = w["lat"][i]
        long = w["long"][i]
              
        
        # Distance to nearest station
        # ACOS( SIN(lat1)*SIN(lat2) + COS(lat1)*COS(lat2)*COS(lon2-lon1) ) * 6371000
        # plain old function doesn't know about rows/columns, it just does its job.
        #def my_func(arg1,arg2):
        #    return arg1+arg2, arg1-arg2  # return multiple responses
        #
        #df['sum'],df['difference'] = 
        #     zip(*df.apply(lambda x: my_func(x['first'],x['second']),axis=1))
        try:
            #df.two = df.two*5
            #print("(", lat, ",", long, ")")

            def calc_distance(slat, slong):
    #                    print("0")
                p1 = math.sin(slat)*math.sin(lat * math.pi/180)
                p2 = math.cos(slat)*math.cos(lat * math.pi/180)*math.cos(long * math.pi/180 - slong)
                d = math.acos(p1 + p2) * 6371000
    #                    print(slat, slong, "-->", d)
                return d

            station["d"] = station.apply(lambda x: calc_distance(x['LATR'], x['LONGR']), axis=1)
            d = station["d"][station["d"] == station.d.min()].values[0]
            sname = station["Train_Station"][station["d"] == station.d.min()].values[0]
            #print("train station ", sname, "distance", d)

            # Distance from central sydney
            # -33.882565° N, 151.20657° E
            d_central = calc_distance(-33.882565 * math.pi/180.0, 151.20657*math.pi/180.0)

        except:
            d = np.NaN
            sname = np.NaN
            d_central = np.Nan
        
        w["station"][i] = d
        w["station_name"][i] = sname
        w["central_station"][i] = d_central
        
    try:
        w = w.drop("longitude")
    except:
        iuu=1
        
    w["date"] = date
    #print(w)
    
    # Let's sort out postcode
    w['region'] = w['postcode'].apply(lambda x: findregion(int(x)))
    
    
    
    
    print(date.strftime("%Y-%m-%d"), "Found ", w.shape[0])
    
    return w

In [ ]:
#resp = req.get("https://www.domain.com.au/auction-results/sydney/2020-09-12")

In [ ]:
#print(findregion(2073))

#df = processpage(resp)
#print(df)
#df.dtypes
#print(df)

In [ ]:
#df["geoLocation"][0] = 10
#df["geoLocation"][1]

In [ ]:
# TRY NUMBER 4\n",
#\n",
# SET DATE OF A SATURDAY TO WORK BACK FROM HERE \n",
#\n",
startdate = datetime.datetime.strptime('12/9/2020', '%d/%m/%Y')
untildate = datetime.datetime.strptime('1/9/2017', '%d/%m/%Y')

df = pd.DataFrame()
date = startdate
while (date > untildate):
    url = "https://www.domain.com.au/auction-results/sydney/" + date.strftime("%Y-%m-%d")
    print(date.strftime("%Y-%m-%d"), "Working on page: ", url)
    resp = req.get(url)
    d = processpage(resp)
    df = pd.concat([df,d],sort=False)
    date -= timedelta(days=7)  
    print(date.strftime("%Y-%m-%d"), "Total now: ", df.shape[0])
 
#df.set_index('index', inplace=True)
print("Got ", df.shape[0], "rows")
#print(df)

In [ ]:
df


In [ ]:
#
#
# Output as CSV
#
file_name="Sydney-List.csv"
df.to_csv(file_name, sep=',', encoding='utf-8')